In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os 
import random
import cv2

In [39]:
categories = {'cane': 'dog', "cavallo": "horse", "gatto": "cat", "mucca": "cow", "pecora": "sheep"}
data=[]
animals=["dog", "horse", "cat", "cow",  "sheep",]
img_size=224
def create_data():
        for category,translate in categories.items():
            path="./raw-img/"+category
            target=animals.index(translate)
            
            i = 0
            for img in os.listdir(path):
                i += 1
                if i <= 500 :
                    try:
                        img_array=cv2.imread(os.path.join(path,img),cv2.IMREAD_COLOR)
                        new_img_array=cv2.resize(img_array,(img_size,img_size))
                        data.append([new_img_array,target])
                    except Exception as e:
                        pass
                else : 
                    break
                
            
create_data()            

In [40]:
random.shuffle(data)
x=[]
y=[]
for features,labels in data:
    x.append(features)
    y.append(labels)   

In [41]:
x = np.array(x)
y = np.array(y)

In [42]:
x.shape

(2500, 224, 224, 3)

In [43]:
# for loading/processing the images  
from keras.preprocessing.image import load_img 
from keras.preprocessing.image import img_to_array 
from keras.applications.vgg16 import preprocess_input 

In [44]:
# models 
from keras.applications.vgg16 import VGG16 
from keras.models import Model

In [45]:
# load model
model = VGG16()
# remove the output layer
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)

In [46]:
feat = []

for img in x :
    reshaped_img = img.reshape(1,224,224,3) 
    imgx = preprocess_input(reshaped_img)
    features = model.predict(imgx, use_multiprocessing=True)
    feat.append(features)
    
feat = np.array(feat)

In [48]:
feat = feat.reshape(-1,4096)
feat.shape

(2500, 4096)

In [50]:
from sklearn.decomposition import PCA

In [59]:
pca = PCA(n_components=50, random_state=22)
pca.fit(feat)
X = pca.transform(feat)

In [60]:
X.shape

(2500, 50)

In [61]:
with open('data.txt' ,"w") as file:
    file.write(str(X.shape[0])+' ')
    file.write(str(X.shape[1])+' ')
    file.write('\n')
    for i in range(X.shape[0]):
        file.write(str(y[i])+' ')
        for j in range(X.shape[1]):
            file.write(str(X[i,j])+' ')
        file.write('\n')